In [2]:
import nest_asyncio

nest_asyncio.apply()

In [3]:
from langroid.agent.chat_agent import ChatAgentConfig
from langroid.language_models.openai_gpt import OpenAIChatModel, OpenAIGPTConfig
from banking_classification import BankingTextClassifier

In [4]:
chat_agent_config = ChatAgentConfig(
    llm = OpenAIGPTConfig(
        chat_model = OpenAIChatModel.GPT4,
    ),
    vecdb = None,
)

In [5]:
# Uncomment only to download dataset and prepare seed and test files

In [6]:
# Download and save banking text classifier into seed and test files
# from datasets import load_dataset
# 
# dataset = load_dataset("banking77")

In [7]:
base_prompt = """
You are an expert at understanding bank customers support complaints and queries.
Your job is to correctly classify the provided input example into one of the following categories.
You will answer with just the the correct output label and nothing else.
Categories:
    activate_my_card,
   age_limit,
   apple_pay_or_google_pay,
   atm_support,
   automatic_top_up,
   balance_not_updated_after_bank_transfer,
   balance_not_updated_after_cheque_or_cash_deposit,
   beneficiary_not_allowed,
   cancel_transfer,
   card_about_to_expire,
   card_acceptance,
   card_arrival,
   card_delivery_estimate,
   card_linking,
   card_not_working,
   card_payment_fee_charged,
   card_payment_not_recognised,
   card_payment_wrong_exchange_rate,
   card_swallowed,
   cash_withdrawal_charge,
   cash_withdrawal_not_recognised,
   change_pin,
   compromised_card,
   contactless_not_working,
   country_support,
   declined_card_payment,
   declined_cash_withdrawal,
   declined_transfer,
   direct_debit_payment_not_recognised,
   disposable_card_limits,
   edit_personal_details,
   exchange_charge,
   exchange_rate,
   exchange_via_app,
   extra_charge_on_statement,
   failed_transfer,
   fiat_currency_support,
   get_disposable_virtual_card,
   get_physical_card,
   getting_spare_card,
   getting_virtual_card,
   lost_or_stolen_card,
   lost_or_stolen_phone,
   order_physical_card,
   passcode_forgotten,
   pending_card_payment,
   pending_cash_withdrawal,
   pending_top_up,
   pending_transfer,
   pin_blocked,
   receiving_money,
   Refund_not_showing_up,
   request_refund,
   reverted_card_payment?,
   supported_cards_and_currencies,
   terminate_account,
   top_up_by_bank_transfer_charge,
   top_up_by_card_charge,
   top_up_by_cash_or_cheque,
   top_up_failed,
   top_up_limits,
   top_up_reverted,
   topping_up_by_card,
   transaction_charged_twice,
   transfer_fee_charged,
   transfer_into_account,
   transfer_not_received_by_recipient,
   transfer_timing,
   unable_to_verify_identity,
   verify_my_identity,
   verify_source_of_funds,
   verify_top_up,
   virtual_card_not_working,
   visa_or_mastercard,
   why_verify_identity,
   wrong_amount_of_cash_received,
   wrong_exchange_rate_for_cash_withdrawal

"""

In [8]:
# import pandas as pd
# 
# train_df = pd.DataFrame(dataset['train'])
# test_df = pd.DataFrame(dataset['test'])

In [9]:
labels = ['activate_my_card',
   'age_limit',
   'apple_pay_or_google_pay',
   'atm_support',
   'automatic_top_up',
   'balance_not_updated_after_bank_transfer',
   'balance_not_updated_after_cheque_or_cash_deposit',
   'beneficiary_not_allowed',
   'cancel_transfer',
   'card_about_to_expire',
   'card_acceptance',
   'card_arrival',
   'card_delivery_estimate',
   'card_linking',
   'card_not_working',
   'card_payment_fee_charged',
   'card_payment_not_recognised',
   'card_payment_wrong_exchange_rate',
   'card_swallowed',
   'cash_withdrawal_charge',
   'cash_withdrawal_not_recognised',
   'change_pin',
   'compromised_card',
   'contactless_not_working',
   'country_support',
   'declined_card_payment',
   'declined_cash_withdrawal',
   'declined_transfer',
   'direct_debit_payment_not_recognised',
   'disposable_card_limits',
   'edit_personal_details',
   'exchange_charge',
   'exchange_rate',
   'exchange_via_app',
   'extra_charge_on_statement',
   'failed_transfer',
   'fiat_currency_support',
   'get_disposable_virtual_card',
   'get_physical_card',
   'getting_spare_card',
   'getting_virtual_card',
   'lost_or_stolen_card',
   'lost_or_stolen_phone',
   'order_physical_card',
   'passcode_forgotten',
   'pending_card_payment',
   'pending_cash_withdrawal',
   'pending_top_up',
   'pending_transfer',
   'pin_blocked',
   'receiving_money',
   'Refund_not_showing_up',
   'request_refund',
   'reverted_card_payment?',
   'supported_cards_and_currencies',
   'terminate_account',
   'top_up_by_bank_transfer_charge',
   'top_up_by_card_charge',
   'top_up_by_cash_or_cheque',
   'top_up_failed',
   'top_up_limits',
   'top_up_reverted',
   'topping_up_by_card',
   'transaction_charged_twice',
   'transfer_fee_charged',
   'transfer_into_account',
   'transfer_not_received_by_recipient',
   'transfer_timing',
   'unable_to_verify_identity',
   'verify_my_identity',
   'verify_source_of_funds',
   'verify_top_up',
   'virtual_card_not_working',
   'visa_or_mastercard',
   'why_verify_identity',
   'wrong_amount_of_cash_received',
   'wrong_exchange_rate_for_cash_withdrawal']

In [10]:
# for idx, row in train_df.iterrows():
#    label_id = row['label']
#    train_df.at[idx, 'label'] = labels[label_id]
# for idx, row in test_df.iterrows():
#    label_id = row['label']
#    test_df.at[idx, 'label'] = labels[label_id]
# train_df = train_df.sample(n=200, replace=False)
# test_df = test_df.sample(n=2000, replace=False)

In [11]:
# train_df.to_csv("seed.csv", index=False)
# test_df.to_csv("test.csv", index=False)

In [12]:
banking_text_classifier = BankingTextClassifier()
banking_text_classifier.__int__(
    chat_agent_config,
    "seed.csv",
    "test.csv",
    base_prompt
)

2023-12-22 21:22:02 - WARNING - REDIS_PASSWORD, REDIS_HOST, REDIS_PORT not set in .env file,
                    using fake redis client
2023-12-22 21:22:02 - WARNING - REDIS_PASSWORD, REDIS_HOST, REDIS_PORT not set in .env file,
                    using fake redis client
2023-12-22 21:22:02 - WARNING - REDIS_PASSWORD, REDIS_HOST, REDIS_PORT not set in .env file,
                    using fake redis client
2023-12-22 21:22:02 - WARNING - REDIS_PASSWORD, REDIS_HOST, REDIS_PORT not set in .env file,
                    using fake redis client
2023-12-22 21:22:03 - WARNING - QDRANT_API_KEY, QDRANT_API_URL env variable must be set to use 
                QdrantDB in cloud mode. Please set these values 
                in your .env file. 
                Switching to local storage at :memory: 
                
2023-12-22 21:22:03 - WARNING - QDRANT_API_KEY, QDRANT_API_URL env variable must be set to use 
                QdrantDB in cloud mode. Please set these values 
                in yo

In [ ]:
banking_text_classifier.run_tweet_emotion_detect()

2023-12-22 21:22:06 - WARNING - REDIS_PASSWORD, REDIS_HOST, REDIS_PORT not set in .env file,
                    using fake redis client
2023-12-22 21:22:06 - WARNING - REDIS_PASSWORD, REDIS_HOST, REDIS_PORT not set in .env file,
                    using fake redis client


Output()

Processing idx: 166


2023-12-22 21:22:06 - WARNING - No chunked docs; cannot use bm25-similarity
2023-12-22 21:22:06 - WARNING - No chunked docs; cannot use bm25-similarity


2023-12-22 21:22:06 - WARNING - No chunked docs; cannot use fuzzy matching
2023-12-22 21:22:06 - WARNING - No chunked docs; cannot use fuzzy matching


...

2023-12-22 21:22:30 - WARNING - No chunked docs; cannot use bm25-similarity
2023-12-22 21:22:30 - WARNING - No chunked docs; cannot use bm25-similarity


2023-12-22 21:22:30 - WARNING - No chunked docs; cannot use fuzzy matching
2023-12-22 21:22:30 - WARNING - No chunked docs; cannot use fuzzy matching


receiving_money

Stats:  N_MSG=2, TOKENS: in=752, out=3, max=1024, ctx=8192, COST: now=$0.0227, cumul=$0.2507

Checkpointing llm responses after idx: 175
Sleeping for a min...


In [ ]:
banking_text_classifier.results